# Spectra for DES Y3

In this notebook we would like to check the Limber approximation for the galaxy clustering angular power spectra of DES Y3.

Below we will also explore a CMB lensing cross-correlation example from Advanced ACT survey DR6 release. 

